In [1]:
from collections import deque
from imutils.video import VideoStream
import numpy as np
import cv2
import imutils
import time

In [12]:
lower = (90, 100, 6)
upper = (125, 255, 255)
buffer = 64 # Max buffer size
pts = deque(maxlen = buffer)

In [13]:
vs = cv2.VideoCapture(0)
time.sleep(2)

In [14]:
while True:
    _, frame = vs.read()
    
    frame = imutils.resize(frame, width=600)
    blurred = cv2.GaussianBlur(frame, (11, 11), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    
    mask = cv2.inRange(hsv, lower, upper)
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)
    
    #cv2.imshow("Frame", mask)
    #key = cv2.waitKey(1) & 0xFF
    
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
                           cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    center = None
    
    if len(cnts) > 0:
        c = max(cnts, key = cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
        
        if radius > 10:
            cv2.circle(frame, (int(x), int(y)), int(radius),
                      (0, 255, 255), 2)
            cv2.circle(frame, center, 5, (0, 0, 255), -1)
    
    pts.appendleft(center)
    
    for i in range(1, len(pts)):
        if pts[i-1] is None or pts[i] is None:
            continue
        
        thickness = int(np.sqrt(buffer/float(i+1))*2.5)
        cv2.line(frame, pts[i-1], pts[i], (0, 0, 255), thickness)
        
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord("q"):
        break
    
vs.release()
cv2.destroyAllWindows()